In [ ]:
from escriptorium_connector import EscriptoriumConnector
from escriptorium_connector.dtos import PostProject, PostDocument, ReadDirection, LineOffset
import os
from dataclasses import dataclass
import json
import datetime

In [ ]:

# eScriptorium instance details
ESCIPTORIUM_URL = 'http://localhost:8080'  # Replace with your instance URL
USERNAME = 'username' # replace with your credentials
PASSWORD = 'password' 

# Initialize the connector
connector = EscriptoriumConnector(ESCIPTORIUM_URL, USERNAME, PASSWORD)

# Initialize the connector using the API token
# API_TOKEN = #Replace with your actual API token
# connector = EscriptoriumConnector(ESCIPTORIUM_URL, api_key=API_TOKEN, project=PROJECT_NAME)

In [ ]:
# create new project

# Create a new project
new_project_name = "My New Project"  # Give your project a name
new_project_name_slug = "my-new-roject"
project_data = PostProject(name=new_project_name, slug=new_project_name_slug)  # The slug is usually a URL-friendly version of the name
new_project = connector.create_project(project_data)

print(f"Created new project: {new_project.name}")


In [ ]:
# initialize an instance from an existing project

PROJECT_NAME = 'Project Name' # replace with project name

existing_project_name_slug = connector.get_project(connector.get_project_pk_by_name(PROJECT_NAME)).slug


In [ ]:
#create a new document

new_document_name = "My real new document" # change
main_script = "Latin" # change as needed


document_data = PostDocument(
    name=new_document_name,
    project=project_slug,
    main_script=main_script,
    read_direction=ReadDirection.LTR, #note 
    line_offset=LineOffset.BASELINE,
    tags=[]
)

# Modified create_document method from EscriptoriumConnector to fit my setup
def modified_create_document(connector, doc_data):
    url = f"{connector.api_url}documents/"
    # Convert the dataclass to a dictionary
    payload = doc_data.__dict__
    print(f"Request URL: {url}")
    print(f"Payload: {payload}")

    # Perform the POST request
    response = connector.http.post(url, json=payload)

    # Print raw response text
    print(f"Raw Response: {response.text}")


# Call the modified create_document method
modified_create_document(connector, document_data)

In [ ]:
# locate your document ID (pk)
connector.get_documents()


In [ ]:
# Upload images to an existing document

IMAGE_FOLDER = "path\to\folder" #"Images"  #

# ID of the document to which the images will be uploaded
DOCUMENT_ID = document_pk  # Replace with your document ID (pk)

@dataclass
class ImageDataInfo:
    filename: str
    # Add other fields here if needed

def upload_images_to_document(connector, folder_path, document_id):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                image_data = file.read()

                # Create an instance of the dataclass
                image_data_info = ImageDataInfo(filename=filename)

                # Upload the image
                #print(f"Uploading {filename} to document {document_id}...")
                response = connector.create_document_part(
                    document_pk=document_id, 
                    image_data_info=image_data_info, 
                    filename=filename, 
                    image_data=image_data
                )
                print(f"Finished uploading {filename}")

                
# Start the upload process
upload_images_to_document(connector, IMAGE_FOLDER, DOCUMENT_ID)

In [ ]:
# upload page XMLs for existing document and images

PAGE_FOLDER = "path\to\folder"

# ID of the document to which the images will be uploaded
DOCUMENT_ID = connector.get_document(7).pk  # Replace with your document ID

def upload_xmls_to_document(connector, folder_path, document_id):
    """Uploads image files to a specified document in eScriptorium."""
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.xml'):  # Add/remove extensions as needed
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as file:
                # Prepare image data for upload
                file_data = file.read()

                # Upload the image
                print(f"Uploading {filename} to document {document_id}...")
                response = connector.upload_part_transcription(
                    document_pk=document_id, 
                    transcription_name = f'transcript {datetime.date.today()}', #change to desired name
                    filename=filename, 
                    file_data=file_data,
                    override='off' #change to on if you want to override existing transciptions
                )
                print(f"Finished uploading {filename}")

                
# Start the upload process
upload_xmls_to_document(connector, PAGE_FOLDER, DOCUMENT_ID)